countvectorizer: 출현 빈돋만 생성하기에 약점을 가지고 있음

tidf를 이용해 많이 나오기는 하지만,
하나의 문장내에서는 단어들에 가중치를 주어 tfidfvectorizer를 준다.

# 자연어 처리에서 특징추출
###독립변수 나올때 특징 추출이 아님(다른 얘기)

여기에서의 특징추출이란 단어나 문장들을 어떤 특징 값으로 변환하는 것을 의미한다.
문자로 구성된 데이터를 모델에 적용할 수 있도록, 특징을 추출해 수치화한다.
단어의 수를 파악해 문장을 분석하는 방법

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

textData = ['나는 배 고프다 아니 배가 고프다.','나는 내일 점심 뭐 먹지?','나는 내일 공부 해야겠다.','나는 점심 먹고 공부 해야지!']
count_vec = CountVectorizer(analyzer='word',ngram_range=(1,1), stop_words=['나는']) # ngram_range : 단어장 생성에 사용할 토큰 크기를 결정함
count_vec.fit(textData)
print(count_vec.get_feature_names_out()) # 전치사를 빼고 싶을 때는 konlpy를 걸어주면 된다.
# 문장에서 공백을 기준으로 단어별로 잘라주고 있음
# 한글자짜리는 제외하고 구두점은 빼주고 있다 -> 단어 사전을 만들고 있음 => vocabulary를 확인해주면 된다.
print(count_vec.vocabulary_)
# {'고프다': 0, '아니': 6, '배가': 5, '내일': 2, '점심': 7, '먹지': 4, '공부': 1, '해야겠다': 8, '먹고': 3, '해야지': 9} -> 가나다 순으로 인덱싱을 함
# dict에 요소는 10개임
print([textData[0]])

sentence=[textData[0]]
print(count_vec.transform(sentence)) #벡터화
print(count_vec.transform(sentence).toarray())
# 각 sentence가 있으면 vocabulary를 만들어주고 토큰 별로 잘라줌

# TF-IDF
Term Fequency(1개의 문서 내 특정 단어 등장빈도) - inverse Document Frequency


## DF- 특정 단어가 나타나는 문장 수
정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다

단순히 빈도수로 그 단어의 가치를 정하는 것이 아니라, 여러문장에 많이 등장하는 단어는 패널티를 주어 단어 빈도의 스케일을 맞추는 기법.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

textData = ['나는 배 고프다 아니 배가 고프다.','나는 내일 점심 뭐 먹지?','나는 내일 공부 해야겠다.','나는 점심 먹고 공부 해야지!']
tfidf_vec = TfidfVectorizer(analyzer='word',ngram_range=(1,1), stop_words=['나는'])
tfidf_vec.fit(textData)
print(tfidf_vec.get_feature_names_out())
print(tfidf_vec.vocabulary_)
print(tfidf_vec.transform(sentence).toarray())
print()

sentence= [textData[3]]
print(sentence)
print(tfidf_vec.transform(sentence))
print(tfidf_vec.transform(sentence).toarray())



# TfidfVectorizer
를 사용해 텍스트(형태소 분석기 Okt를 적용)를 벡터로 변환 후 단어 간 유사도 계산

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

okt=Okt()

def tokenizeFunc(ss): # 형태소(일정한 의미가 있는 작은 단위) 분석용 수행 함수 - Okt가 해줌
    ss=okt.normalize(ss) #Okt는 정규화를 일부 지원함. 예)'사랑햌' -> '사랑해'로 수정
    ss = okt.morphs(ss) # 형태소 단위로 분리, 반환형은 리스트
    return ss

texts = ['길동이는 파이썬을 좋아합니다','길동이는 웹을 잘 합니다','길동이는 운동을 매우 잘합니다'] # 스포츠와 관련된 내용
new_texts = ['길동이는 파이썬을 좋아하고 운동을 잘합니다.'] #글은 어느 장르에 속하는 글이다.
# 시계열 데이터가 아님, 순서는 상관이 없음

tfidf = TfidfVectorizer(tokenizer=tokenizeFunc, token_pattern=None).fit(texts)
print(tfidf.vocabulary_)
tfidf_matrix = tfidf.fit_transform(texts)
print(tfidf_matrix)
print(tfidf_matrix.toarray()) # 같은 빈도라도 tfidf값이 높게 나온것


In [ ]:
for ntext in new_texts:
    tftrans = tfidf.transform([ntext]) #새로운 문장을 벡터로 변환함 ['길동이는 파이썬을 좋아하고 운동을 잘합니다.']
    print('tftrans : \n',tftrans)
    # 새로운 문장과 기존 문장들 사이의 코사인 유사도(데이터 크기차이에 관계가 없이 계산 가능) 계산
    cosine_simil = cosine_similarity(tftrans,tfidf_matrix)
    print('cosine_simil : ', cosine_simil) # [[0.6294     0.65051252 0.77272178]]

    #출력
    print(f'새로운 문장 : {ntext}')
    print('----------'*10)
    print(f'기존 문장 : ')
    for idx in range(3):
        print(cosine_simil.argsort()[0]) # [0]은 2차원 배열의 0행
        print((idx+1)*-1)
        print(cosine_simil[0][(idx+1)*-1])
        most_simil_idx=cosine_simil.argsort()[0][(idx+1)*-1]
        print(most_simil_idx)
        most_simil_sentence = texts[most_simil_idx]
        simil_score = cosine_simil[0][most_simil_idx]
        print(f'{most_simil_sentence}(유사도:{simil_score:.3f})')
    print()
# 새로운 문장과 기존 문장과의 유사도가 있다 - 유사도가 높은 것을 토대로 판단한다.
# Dense란? 자연어 처리는 순서가 있다 순서가 있는 자연어를 가지고 문장 생성 또는 번역 또는 챗봇 요약 문서에 이미지에 설명 달기 주식 에측 이런 것들에 소리로 번역, 예약 시스템 안내 시스템 등을 할 수 있다. 자연스럽게 바뀔수 있음
